# ETL del dataset homicidios
Importación de librerías 

In [27]:
import pandas as pd
import numpy as np

## Homicidios hechos

Carga del dataset

In [28]:
df_hechos = pd.read_excel('../Data/homicidios.xlsx', sheet_name = 'HECHOS', na_values=['SD','sd'])

Visualización de las primeras 5 filas

In [29]:
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Información de las columnas del dataset

In [30]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

#### Elimino columnas innecesarias


Luego de analizar las columnas se determino que: 
* Las columnas `AAAA`,`MM`,`DD` hacen referencia a la información de la columna `FECHA`
* La columna `HORA` hacen referencia a la información de la columna `HH`
* Las columnas `LUGAR_DEL_HECHO`, `Calle`, `Altura`, `Cruce` ,`Dirección Normalizada` y `XY (CABA)` ya que con usar las columnas **pos x** y **pos y** se sabe la latitud y longitud respectiva del accidente
* La columna `PARTICIPANTES` hace referencia al contenido de las columnas **victima** y **acusado**

In [31]:
df_hechos.drop(columns=['AAAA','MM','DD','HORA','LUGAR_DEL_HECHO','Calle','Altura','Cruce','Dirección Normalizada','XY (CABA)','PARTICIPANTES'],inplace=True)

#### Valores nulos


Conteo de nulos

In [32]:
df_hechos.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
pos x             0
pos y             0
VICTIMA           9
ACUSADO          23
dtype: int64

Extraigo varias muestras de forma aleatoria para verificar otro tipo de nulos

In [33]:
df_hechos.sample(10)

,ID,N_VICTIMAS,FECHA,HH,TIPO_DE_CALLE,COMUNA,pos x,pos y,VICTIMA,ACUSADO
314,2018-0040,1,2018-04-27,11.0,CALLE,13,-58.45686132,-34.57305993,PEATON,CARGAS
684,2021-0086,1,2021-11-25,10.0,AVENIDA,6,-58.44864864,-34.62352826,PEATON,PASAJEROS
655,2021-0057,1,2021-07-10,23.0,AUTOPISTA,2,-58.39257023,-34.57563957,AUTO,OBJETO FIJO
92,2016-0119,1,2016-09-04,6.0,AVENIDA,12,-58.49659234,-34.56353635,NaN,NaN
285,2018-0011,1,2018-01-19,10.0,AVENIDA,2,-58.39092703,-34.59088447,PEATON,PASAJEROS
114,2016-0145,1,2016-11-07,20.0,AVENIDA,13,-58.44523595,-34.55953855,MOTO,AUTO
645,2021-0047,1,2021-06-02,16.0,AVENIDA,4,-58.42190745,-34.65633192,MOTO,AUTO
369,2018-0095,1,2018-08-26,0.0,AVENIDA,9,-58.46739825,-34.65117757,AUTO,AUTO
172,2017-0037,1,2017-03-31,3.0,AVENIDA,1,-58.38896772,-34.61799615,MOTO,CARGAS
272,2017-0159,1,2017-12-20,4.0,AVENIDA,1,-58.38896772,-34.61799615,MOTO,AUTO


Después de extraer varias veces muestras del df_hechos se descubrió que aunque en un principio los valores nulos que se pueden contar son solo 3, hay valores atípicos que se consideran nulos como el `.` en las columnas **pos x** y **pos y** y el `0` en **COMUNA** ya que debería ir desde el 1 al 15 según lo indica el diccionario del df_hechos, por ende se decide remplazar estos por valores nulos para un mejor tratamiento y uso de los datos

In [34]:
# Reemplazo de la Comuna 0 por Valores Nulos
df_hechos.loc[df_hechos['COMUNA'] == 0, 'COMUNA'] = np.nan
#Reemplazo de los . por valores nulos de las columnas pos x y pos y

df_hechos.loc[df_hechos['pos x'] == '.' ,'pos x'] = np.nan
df_hechos.loc[df_hechos['pos y'] == '.' ,'pos y'] = np.nan

In [35]:
df_hechos.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            2
pos x            12
pos y            12
VICTIMA           9
ACUSADO          23
dtype: int64

#### Cambio de tipo des de datos



Las columnas `HH`, `TIPO_DE_CALLE`, `COMUNA` y `ACUSADO` serán transformadas al tipo de dato categórico <br>
Las columnas `pos X` y `pos Y` serán transformadas al tipo de dato flotante

In [36]:
# cambio a tipo categórico
df_hechos['HH'] = df_hechos['HH'].astype('category')
df_hechos['TIPO_DE_CALLE'] = df_hechos['TIPO_DE_CALLE'].astype('category')
df_hechos['COMUNA'] = df_hechos['COMUNA'].astype('category')
df_hechos['ACUSADO'] = df_hechos['ACUSADO'].astype('category')


# cambio a flotante los valores de pos x y pos y
df_hechos['pos x'] = df_hechos['pos x'].astype('float')
df_hechos['pos y'] = df_hechos['pos y'].astype('float')

#### Columnas

##### Renombrar columnas
Se modifican las columnas para mejor lectura y un futuro tratamiento mas cómodo

In [37]:
# cambio las columnas a minúsculas
df_hechos.columns = df_hechos.columns.str.lower()
# renombro columnas
df_hechos.rename(columns={'ID': 'id_hecho', 'pos x': 'longitud', 'pos y': 'latitud','tipo_de_calle':'tipo_calle'},inplace=True)

##### Se crea la columna  `gravedad` y se asigna el valor 'FATAL'

In [38]:
df_hechos['gravedad'] = 'FATAL'

In [39]:
df_hechos.head()

,id,n_victimas,fecha,hh,tipo_calle,comuna,longitud,latitud,victima,acusado,gravedad
0,2016-0001,1,2016-01-01,4.0,AVENIDA,8.0,-58.475340,-34.687570,MOTO,AUTO,fatal
1,2016-0002,1,2016-01-02,1.0,GRAL PAZ,9.0,-58.508775,-34.669777,AUTO,PASAJEROS,fatal
2,2016-0003,1,2016-01-03,7.0,AVENIDA,1.0,-58.390403,-34.631894,MOTO,AUTO,fatal
3,2016-0004,1,2016-01-10,0.0,AVENIDA,8.0,-58.465039,-34.680930,MOTO,NaN,fatal
4,2016-0005,1,2016-01-21,5.0,AVENIDA,1.0,-58.387183,-34.622466,MOTO,PASAJEROS,fatal


#### Guardo en un CSV para trabajar más cómodo  

In [59]:
df_hechos.to_csv('../Data/homicidios_hechos.csv', index=False)

# Homicidios victimas

Carga del dataset

In [41]:
df_victimas = pd.read_excel('../Data/homicidios.xlsx', sheet_name = 'VICTIMAS', na_values=['SD','sd'])

In [42]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


Información de las columnas del dataset

In [43]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


#### Elimino columnas innecesarias


Luego de analizar las columnas se determino que: 
* Las columnas `AAAA`,`MM`,`DD` hacen referencia a la información de la columna `FECHA`
* La columna `FECHA_FALLECIMIENTO` no es relevante para el análisis que se planea

In [44]:
df_victimas.drop(columns=['AAAA','MM','DD','FECHA_FALLECIMIENTO'],inplace=True)

#### Valores nulos
Conteo de nulos

In [45]:
df_victimas.isnull().sum()

ID_hecho     0
FECHA        0
ROL         11
VICTIMA      9
SEXO         6
EDAD        53
dtype: int64

Extraigo varias muestras de forma aleatoria para verificar otro tipo de nulos

In [46]:
df_victimas.sample(10)

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD
94,2016-0122,2016-09-13,CICLISTA,BICICLETA,MASCULINO,49.0
473,2019-0038,2019-05-04,PEATON,PEATON,FEMENINO,42.0
90,2016-0116,2016-09-02,CONDUCTOR,MOTO,MASCULINO,30.0
695,2021-0076,2021-10-16,CONDUCTOR,MOTO,MASCULINO,31.0
9,2016-0013,2016-02-10,PEATON,PEATON,MASCULINO,16.0
539,2020-0001,2020-01-04,PEATON,PEATON,FEMENINO,87.0
144,2016-0177,2016-12-29,PEATON,PEATON,FEMENINO,27.0
699,2021-0080,2021-10-28,CONDUCTOR,MOTO,MASCULINO,32.0
374,2018-0085,2018-08-03,CONDUCTOR,MOTO,MASCULINO,30.0
61,2016-0083,2016-06-28,PEATON,PEATON,FEMENINO,76.0


#### Cambio de tipo des de datos



Las columnas `ROL`, `VICTIMA` y `SEXO` serán transformadas al tipo de dato categórico


In [47]:
# cambio a formato categórico
df_victimas['ROL'] = df_victimas['ROL'].astype('category')
df_victimas['VICTIMA'] = df_victimas['VICTIMA'].astype('category')
df_victimas['SEXO'] = df_victimas['SEXO'].astype('category')

#### Columnas

##### Renombrar columnas
Se modifican las columnas para mejor lectura y un futuro tratamiento mas cómodo

In [48]:
# cambio las columnas a minúsculas
df_victimas.columns = df_victimas.columns.str.lower()
# renombro columnas
df_victimas.rename(columns={'edad': 'edad_victima','victima':'vehiculo_victima','sexo':'sexo_victima'},inplace=True)

Remplazo de la columna `rol` *PASAJERO_ACOMPAÑANTE* por `PASAJERO`

In [49]:
df_victimas['rol'].replace('PASAJERO_ACOMPAÑANTE','PASAJERO',inplace=True)

##### Se crea la columna  `gravedad` y se asigna el valor `FATAL`

In [50]:
df_victimas['gravedad'] = 'FATAL'

In [51]:
df_victimas.head()

,id_hecho,fecha,rol,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,FATAL
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70.0,FATAL
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30.0,FATAL
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18.0,FATAL
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29.0,FATAL


#### Guardo en un CSV para trabajar más cómodo  

In [52]:
df_victimas.to_csv('../Data/homicidios_victimas.csv', index=False)